# Bronze Layer: Configurações

## Bibliotecas

In [0]:
import uuid
import gc
import urllib.request
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, TimestampType, LongType, DateType, TimestampType, BooleanType, DoubleType, ByteType

## Configuração da estrutura do Unity catalog

* data_ex (catalog) 
    * metadados (schema)
        * meta_bronze (table)
        * meta_silver (table)
        * meta_gold (table)
    * lhdw (schema) 
        * versao001 (volumes)
        * versao002 (volumes)
        * versao003 (volumes)       
    * bronze (schema)
        * bronze_log_carga (table)
        * versao003 (volume)
            * bronze_dim_categoria_produto
            * bronze_dim_cliente 
            * bronze_dim_data 
            * bronze_dim_localidade
            * bronze_dim_produto 
            * bronze_fato_vendas  
    * silver (schema)
        * versao* (volume)
            * silver_dim_categoria_produto
            * silver_dim_cliente 
            * silver_dim_data 
            * silver_dim_localidade
            * silver_dim_produto 
            * silver_fato_vendas
    * gold  (schema)
        * versao* (volume)
            * dim_categoria_produto
            * dim_cliente 
            * dim_data 
            * dim_localidade
            * dim_produto 
            * fato_vendas

In [0]:
# Criação do catalog
spark.sql("create catalog if not exists data_ex")

# Criação do schema
spark.sql("create schema if not exists data_ex.metadados")
spark.sql("create schema if not exists data_ex.lhdw")
spark.sql("create schema if not exists data_ex.bronze")
spark.sql("create schema if not exists data_ex.silver")
spark.sql("create schema if not exists data_ex.gold")

# Criação das tabelas de metadados fixos, para controle de job 
spark.sql("""create table if not exists data_ex.metadados.meta_bronze (
    id_job string
) using delta """)

spark.sql("""create table if not exists data_ex.metadados.meta_silver (
    id_job string
) using delta """)

spark.sql("""create table if not exists data_ex.metadados.meta_gold  (
    id_job string
) using delta """)

# Abaixo são geradas uuids do tipo 4, a qual não requer argumentos e é gerada aleatoriamente
id_job_bronze = str(uuid.uuid4())
id_job_silver = str(uuid.uuid4())
id_job_gold = str(uuid.uuid4())

# Armazena a uuid gerada na respectiva camada
spark.sql(f"insert into data_ex.metadados.meta_bronze values ('{id_job_bronze}')")
spark.sql(f"insert into data_ex.metadados.meta_silver values ('{id_job_silver}')")
spark.sql(f"insert into data_ex.metadados.meta_gold values ('{id_job_gold}')")

## Configuração do servidor para trabalho no Bronze Layer


In [0]:
# Otimização retirada do material disponibilizado do curso de Databricks
spark = SparkSession.builder \
    .appName("Load Data Bronze") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Define um número fixo de partições para shuffle, melhorando o paralelismo                 
# Define o tamanho máximo de partições para evitar muitos arquivos pequenos        
# Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita    
# Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados

## Funções do Bronze Layer

### download()
Retorna o nome do volum em que o download foi feito

In [0]:
# Função para baixar os arquivos
def download_dataset(csv_files, url, path):
    # Tenta realizar o download dos arquivos
    try :
        # Variável para armazenar o nome do novo volume
        volume = ""
        try:
            # Conta quantos volumes já existem dentro desse do schema
            numero = spark.sql("show volumes in data_ex.lhdw").count()
            # Cria o comando sql para criar um novo volume para a nova versão do download
            volume = f"download{numero+1:03d}"
            sql_command = f"create volume if not exists data_ex.lhdw.{volume}"
            # Executa o comando sql acima
            spark.sql(sql_command)
        # Lança um exceção caso houver um erro
        except Exception as e:
            print(f"Erro ao criar o volume: {e}")
        for file in csv_files:
            down_path = f"{url+file}"
            up_path = f"{path+volume}/{file}"
            urllib.request.urlretrieve(down_path, up_path)
            print(f"Csv {file} baixado com sucesso!")
    # Caso não consiga realizar o download, uma exceção é lançada
    except Exception as e:
        print(f"Erro ao baixar o arquivo: {file} - {e}")
    return volume

### read_dim()
Função para ler as dimensões

In [0]:
def read_dim(path, file, schema):
    try:
        df = spark.read\
            .option("header", "true")\
            .schema(schema)\
            .csv(f"{path}/{file}")
        return df
    except:
        return 0

### save_dim()

In [0]:
def save_dim(df_read, dime, camada):
    try:
        df_fato = df_read.write.format("delta").mode("overwrite").save(f"{camada}/{dime}")
        return 1
    except Exception as e:
        return 0

### read_fato()

In [0]:
def read_fato(path, files, schema, n):
    print("entrou na read_fato()")
    try:
        vendas_df = [0]
        # Aplica um loop para ler todos os arquivos vendas
        for i in range(0, n):
            try:
                df = spark.read.option("header", "true")\
                    .schema(schema_vendas)\
                    .csv(f"{path}/{files[i]}")
                vendas_df[0] = vendas_df[0] + df.count()
                vendas_df.append(df)
            except Exception as e:
                print(f"Erro ao ler o arquivo {path}/{files[i]}: {e}")
        return vendas_df
    except Exception as e:
        print(f"Erro ao ler o arquivo {files[i]}: {e}")
    return 0

### save_fato()

In [0]:
def save_fato(df, fato, n, camada):
    #Loop para criar a tabela de fato vendas
    for i in range(0, n):
        try:
            df[i].write.format("delta").mode("append").save(f"{camada}/{fato}")
        except Exception as e:
            print(f"Erro ao salvar o dataframe vendas: {e}")
            return 0
    return 1

### makeLogTable()

In [0]:
def makeLogTable():
    n = 0
    try:
        n = spark.sql("show tables in data_ex.bronze").count()
    except:
        print("ERRO ao ler a tabela de logs")
    if n == 0:
        try :
            spark.sql("""create table if not exists data_ex.bronze.bronze_log_carga (
                id_carga string,
                id_job string,
                nome_arquivo string,
                fonte string,
                camada string,
                path_origem string,
                path_destino string,
                data_inicio timestamp,
                data_fim timestamp,
                duracao_ms string,
                registros_lidos integer,
                registros_gravados integer,
                status string,
                mensagem_erro string,
                data_execusao date
                ) using delta """)
        except Exception as e:
            print(f"Erro ao criar a tabela de logs: {e}")
    elif n > 1:
        print(f"Erro na árvore de diretórios, exite mais de {n} tabelas de logs na camada bronze")
    else:
        print("Tabela de logs já existe na camada bronze")

### generate_pre_log()

In [0]:
def generate_pre_log(file, dime, volume_salvo):
  try:
    id_carga = str(uuid.uuid4())
    id_job   = spark.sql("select * from data_ex.metadados.meta_bronze").collect()[0][0]
    nome_arquivo = dime
    fonte = "filesystem_local"
    camada = "bronze"
    path_origem = f"/Volumes/data_ex/lhdw/{volume_salvo}/{file}"
    path_destino = f"/Volumes/data_ex/bronze/{volume_salvo}/{dime}"
    data_inicio = spark.range(1).select(current_timestamp()).collect()[0][0]
    data_fim = ""
    dureação_ms = ""
    registros_lidos = ""
    registros_gravados = ""
    status = "Running"
    mensagem_erro = "null"
    data_execusao = spark.range(1).select(current_date()).collect()[0][0]
    return [id_carga, id_job, nome_arquivo, fonte, camada, path_origem, path_destino, data_inicio, data_fim, dureação_ms, registros_lidos, registros_gravados, status, mensagem_erro, data_execusao]
  except Exception as e:
    return 0

### gerarLog(log_data)

In [0]:
def gerarLog(log_data):
    try:
        log_data[12] = "True"   # Atualiza o status do log
        data_fim = spark.range(1).select(current_timestamp()).collect()[0][0]
        log_data[8] = data_fim  
        duracao_ms = (data_fim - log_data[7]).total_seconds() * 1000
        log_data[9] = duracao_ms
        acao = storeLog(log_data)
        if acao == 1:
            return 1
        else:
            return 0 
    except Exception as e:
        print(f"Erro na função gerarLog(): {e}")

### storeLog()

In [0]:

def storeLog(log_data):
    try:
        row = Row(
            id_carga = log_data[0],
            id_job = log_data[1],
            nome_arquivo = log_data[2],
            fonte = log_data[3],
            camada = log_data[4],
            path_origem = log_data[5],
            path_destino = log_data[6],
            data_inicio = log_data[7],
            data_fim = log_data[8],
            duracao_ms = log_data[9],
            registros_lidos = log_data[10],
            registros_gravados = log_data[11],
            status = log_data[12],
            mensagem_erro = log_data[13],
            data_carga = log_data[14]
        )

        df_log = spark.createDataFrame([row])
        df_log.write.mode("append").insertInto("data_ex.bronze.bronze_log_carga")
        return 1
    except Exception as e:
        print(f"Erro ao gravar o log: {e}")
        return 0

### Funções de leitura e escrita 

In [0]:
# Criação do Spark DataFrame, leitura dos csv, aplicação dos schemas e escrita em parquet 
def bronze_work_dim(csv_files, schemas, dime, camada, n):
  files = csv_files[:-4]
  # Índice para registrar o nome da dimensão a ser salvada
  for i, (file, schema) in enumerate(zip(files, schemas)):
    # Tenta ler os arquivos csv em lhdw_path
    try:
      # Inicia o registo de logs
      try: 
        log_data = generate_pre_log(file, dime[i], volume_salvo)
        if log_data != 0: 
          # Se schema não for vendas, aplica o schema correspondente a file
          if schema != schema_vendas:
            # Tenta chamar a função para ler um arquivo de não vendas
            df_read = read_dim(lhdw_path, file, schema)
            if df_read == 0:
              printf(f"Erro ao ler o arquivo {file}")
            else:
              # Lê o arquivo csv
              registros_lidos = df_read.count()
              log_data[10] = registros_lidos
              # Silver/Gold process

              # Salva
              save = save_dim(df_read, dime[i], camada)
              if save == 0:
                printf(f"Erro ao salvar o dataframe ")
              else:
                # Salvamento concluído, armazenando o log respectivo
                registros_gravados = spark.read.format("delta").load(f"{camada}/{dime[i]}").count()
                log_data[11] = registros_gravados
                try:
                  acao = gerarLog(log_data)
                  if acao == 1:
                    print(f"DataFrame salvo com sucesso em {camada}/{dime[i]}")
                  else:
                    printf("ERRO ao gravar o log")
                except Exception as e:
                  print(f"Erro ao gerar o log e armazená-lo, função gerarLog(): {e}")
          else:
            print("Não foi possível gerar o pre_log")
        else:
          printf("ERRO ao gerar log")
      except Exception as e:
        print(f"Erro ao chamar o log: {e}")
    except Exception as e:
      print(f"Erro ao iniciar a função de leitura")

In [0]:
def bronze_work_fato(csv_files, schemas, camada, fato_name, volume_salvo, n):
    try:
        log_data = generate_pre_log("vendas", fato_name, volume_salvo)
        # Chama a função de leitura
        df_read = read_fato(lhdw_path, csv_files[-4:] ,schema_vendas, n)

        if df_read == 0:
            print(f"Erro ao ler os arquivos de vendas")
        else:
            registros_lidos = df_read[0]
            log_data[10] = registros_lidos
            df_read.pop(0)
            try:
                # Salvar
                df_fato = save_fato(df_read, fato_name, n, camada)
            except Exception as e:
                printf(f"Erro ao chamar a função save_fato() : {e}")
            if df_fato == 0:
                printf(f"Erro ao salvar o dataframe ")
            else:
                # Salvamento concluído, armazenando o log respectivo
                registros_gravados = spark.read.format("delta").load(f"{camada}/{fato_name}").count()
                log_data[11] = registros_gravados
                acao = gerarLog(log_data)
    except Exception as e:
                print(f"Erro ao ler o arquivo!!: {e}")

## Chamada das funções

### Importação e definição de variáveis

#### Importação do dataset

In [0]:
# Lista de .csv a serem lidos
csv_files = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

# Reposotório do dataset para o desafio
url = "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/"

# Path para o schema de armazenamento, sem o volume
lhdw_path = "/Volumes/data_ex/lhdw/"

# Realiza o download dos arquivos e recebe o nome do volume salvo
volume_salvo = download_dataset(csv_files, url, lhdw_path)



In [0]:
print(volume_salvo)

#### Atualizando os caminhos com o volume obtido e definido os nomes das tabelas que serão salvas no Bronze Layer


In [0]:
# Definição dos diretórios de leitura
lhdw_path = f"{lhdw_path}/{volume_salvo}"

sql_command = f"create volume if not exists data_ex.bronze.{volume_salvo}"
# Criação do volume no schema bronze
spark.sql(sql_command)

# Definição do path bronze
bronze_path = f"/Volumes/data_ex/bronze/{volume_salvo}"

# Lista de nomes das dimensões bronze
dim_name = [
  "bronze_dim_categoria_produto",
  "bronze_dim_cliente",
  "bronze_dim_data", 
  "bronze_dim_localidade",
  "bronze_dim_produto", 
]

# Nome da tabela fato
fato_name = "bronze_fato_vendas"

# Numero de csv que fazem parte da bronze fato
n_fatos = 4


#### Definição manual dos schemas 

In [0]:
# Lista dos nomes dos schemas em ordem de execução
schemas = []

# Definição manual do schemas a serem aplicados nos arquivos csv (futuros parquets)
schema_categoria = StructType([
    StructField("categoria_id", LongType(), True),
    StructField("categoria_nome", StringType(), True),
])
schemas.append(schema_categoria)

schema_cliente = StructType([
    StructField("cliente_id", LongType(), True),
    StructField("nome_cliente", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True)
])
schemas.append(schema_cliente)

schema_data = StructType([
    StructField("data_id", LongType(), True),
    StructField("data", DateType(), True),
    StructField("ano", IntegerType(), True),
    StructField("mes", IntegerType(), True),
    StructField("dia", IntegerType(), True),
    StructField("dia_semana", StringType(), True), 
    StructField("final_de_semana", ByteType(), True)
])
schemas.append(schema_data)

schema_localidade = StructType([
    StructField("localidade_id", LongType(), True),
    StructField("estado", StringType(), True),
    StructField("cidade", StringType(), True),
    ])  
schemas.append(schema_localidade)

schema_produto = StructType([
    StructField("produto_id", LongType(), True),
    StructField("preco_lista", DoubleType(), True),
    StructField("categoria_nome", StringType(), True)
])
schemas.append(schema_produto)

schema_vendas = StructType([
    StructField("venda_id", LongType(), True),
    StructField("cliente_id", LongType(), True),
    StructField("produto_id", LongType(), True),
    StructField("data_id", LongType(), True),
    StructField("categoria_id", LongType(), True),
    StructField("localidade_id", LongType(), True),
    StructField("quantidade", LongType(), True),
    StructField("preco_lista", DoubleType(), True),
    StructField("valor_total", DoubleType(), True)
])
schemas.append(schema_vendas)

### Operações coma as funções

In [0]:
# Garantindo a existencia da Bronze log table
makeLogTable()

In [0]:
# Executando a função principal da dim
bronze_work_dim(csv_files, schemas, dim_name, bronze_path, 4)


In [0]:
# Executando os fatos
bronze_work_fato(csv_files, schemas, bronze_path, fato_name, volume_salvo, 4)

In [0]:
display(spark.sql("select * from data_ex.bronze.bronze_log_carga"))

## Limpar memória

In [0]:
gc.collect()

# Reset 

In [0]:
%skip
spark.sql("drop schema if  exists data_ex.lhdw cascade")
spark.sql("drop schema if  exists data_ex.bronze cascade")
spark.sql("drop schema if  exists data_ex.metadados cascade")
spark.sql("drop schema if  exists data_ex.silver cascade")
spark.sql("drop schema if  exists data_ex.gold cascade")
spark.sql("drop table if exists data_ex.metadados.meta_bronze")
spark.sql("drop table if exists data_ex.metadados.meta_silver")
spark.sql("drop table if exists data_ex.metadados.meta_gold")

## Extra: Printar tabelas bronze

In [0]:

bronze_dim_categoria_produto = spark.read.format("delta").load(f"/Volumes/data_ex/bronze/{volume_salvo}/bronze_dim_categoria_produto")
display(bronze_dim_categoria_produto)

bronze_dim_cliente = spark.read.format("delta").load(f"/Volumes/data_ex/bronze/{volume_salvo}/bronze_dim_cliente")
display(bronze_dim_cliente)

bronze_dim_data = spark.read.format("delta").load(f"/Volumes/data_ex/bronze/{volume_salvo}/bronze_dim_data")
display(bronze_dim_data)

bronze_dim_localidade = spark.read.format("delta").load(f"/Volumes/data_ex/bronze/{volume_salvo}/bronze_dim_localidade")
display(bronze_dim_localidade)

bronze_dim_produto = spark.read.format("delta").load(f"/Volumes/data_ex/bronze/{volume_salvo}/bronze_dim_produto")
display(bronze_dim_produto)

bronze_fato_vendas = spark.read.format("delta").load(f"/Volumes/data_ex/bronze/{volume_salvo}/bronze_fato_vendas")
display(bronze_fato_vendas)